In [2]:
from astropy.modeling import models, fitting
from dendrocat import RadioSource
from astropy.io import fits
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import sys
import importlib
from astropy.wcs import WCS
from scipy import interpolate
import pandas as pd
import sympy as syp
from astropy import units as u
import radio_beam
from astropy.table import Table
from radio_beam import Beam
from astropy import coordinates
from astropy import wcs
from astropy.nddata.utils import Cutout2D
#from dendrocat.aperture import Ellipse
from matplotlib.patches import Rectangle, Ellipse
import regions
from scipy.optimize import curve_fit
from regions import EllipseAnnulusSkyRegion, EllipseAnnulusPixelRegion

W51 = '/orange/adamginsburg/w51/TaehwaYoo/'
W51b6 = '/orange/adamginsburg/w51/TaehwaYoo/2015.1.01596.S_W51_B6_LB/continuum_images/'
W51cont='/orange/adamginsburg/w51/TaehwaYoo/b6contfits/'
w51e2_b6_briggs=W51cont+'W51e2_cont_bigbriggs.image.fits'
w51e2_b6_robust0=W51cont+'W51e2_cont_big_robust0.image.fits'
w51e2_b6_uniform=W51cont+'W51e2_cont_biguniform.image.fits'
w51e2_b6_superuniform=W51cont+'W51e2_cont_bigsuperuniform.image.fits'

w51n_b6_briggs = W51cont+'W51n_cont_bigbriggs.image.fits'
w51n_b6_robust0 = W51cont+'w51n_cont_big_robust0.image.fits'
w51n_b6_uniform = W51cont+'W51n_cont_biguniform.image.fits'
w51n_b6_superuniform = W51cont+'W51n_cont_bigsuperuniform.image.fits'
w51n_b6_natural = W51cont+'W51n_cont_bignatural.image.fits'

W51b3 = '/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/'

w51n_b3_tt0 = W51b3+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b3_tt1 = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51n_b3_alpha = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51conv = '/orange/adamginsburg/w51/TaehwaYoo/convolved_new/'
w51n_b6_conv = w51conv + 'w51n_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b3_tt0 = W51b3+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51e_b3_tt1 = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51e_b3_alpha = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51e_b6_conv_old = w51conv + 'w51e2_cont_bigbriggs.image.convB3_briggs.fits'
w51e_b6_conv = w51conv + 'w51e_new_B6_conv.fits'
w51n_b6_conv = w51conv + 'w51n_new_B6_conv.fits'

w51e_b6_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B6_conv.fits'
w51e_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B3_conv.fits'


w51n_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51n_B3_conv.fits'

w51n_b6_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'

w51n_b6_conv_old = w51conv + 'w51n_B6_conv.fits'
w51n_b3_conv_old = w51conv + 'w51n_B3_conv.fits'
w51e_b3_conv_old = w51conv + 'w51e_B3_conv.fits'
w51e_b6_conv_old = w51conv + 'w51e_B6_conv.fits'



w51e_dendro_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51e_matched.fits'
w51n_dendro_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51n_matched.fits'

w51e_b6_background = '/home/t.yoo/w51/catalogue/dendrogram/background_noise_w51e_b6_pb.dat'
w51e_b3_background = '/home/t.yoo/w51/catalogue/dendrogram/background_noise_w51e_b3_pb.dat'
w51n_b6_background = '/home/t.yoo/w51/catalogue/dendrogram/background_noise_w51n_b6_pb.dat'
w51n_b3_background = '/home/t.yoo/w51/catalogue/dendrogram/background_noise_w51n_b3_pb.dat'

w51e_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51n.spw0thru19.14500.robust0.thr0.1mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'




In [6]:
hdrNB3 = fits.getheader(w51e_b3_tt0)  
hdrNB6 = fits.getheader(w51e_b6_calibrated_pbcor)  
wcsNB3 = WCS(hdrNB3,naxis=2)
wcsNB6 = WCS(hdrNB6,naxis=2)

scaleNB6 = wcsNB6.proj_plane_pixel_scales()[0]
scaleNB3 = wcsNB3.proj_plane_pixel_scales()[0]

my_beamNB6 = Beam.from_fits_header(hdrNB6)
my_beamNB3 = Beam.from_fits_header(hdrNB3)
sig_to_fwhm = np.sqrt(2*np.log(2))*2
fwhm_dec_b6 =sig_to_fwhm*np.sqrt((my_beamNB6.major.value/scaleNB6.value/sig_to_fwhm+0.05) * (my_beamNB6.minor.value/scaleNB6.value/sig_to_fwhm+0.05) - my_beamNB6.major.value/scaleNB6.value/sig_to_fwhm*my_beamNB6.minor.value/scaleNB6.value/sig_to_fwhm)

fwhm_dec_b3 =sig_to_fwhm*np.sqrt((my_beamNB3.major.value/scaleNB3.value/sig_to_fwhm+0.05) * (my_beamNB3.minor.value/scaleNB3.value/sig_to_fwhm+0.05) - my_beamNB3.major.value/scaleNB3.value/sig_to_fwhm*my_beamNB3.minor.value/scaleNB3.value/sig_to_fwhm)

print(fwhm_dec_b6*scaleNB6.value*np.pi/180*5.41*1000*206265)
print(fwhm_dec_b3*scaleNB3.value*np.pi/180*5.41*1000*206265)
print(fwhm_dec_b6*scaleNB6.value*np.pi/180*5.1*1000*206265)
print(fwhm_dec_b3*scaleNB3.value*np.pi/180*5.1*1000*206265)


25.6708090410984
51.22030297449857
24.199838467578896
48.28531334010032
